In [1]:
try:
    import google.colab
    !rm -rf /content/SportWarzSim
    !git clone https://github.com/daltontf/SportWarzSim.git 
    %cd SportWarzSim
except ImportError:
    pass

In [2]:
from sportwarz import LeaguesModel

leagues_model = LeaguesModel()

leagues_model.load_counties_geojson()

In [3]:
leagues_model.load_counties_data()

In [4]:
leagues_model.load_leagues(["NFL", "NBA", "MLB", "NHL", "MLS"])

In [5]:
leagues_model.calculate_distances() 

In [6]:
leagues_model.compute_shares()  

In [7]:
leagues_model.compute_output_dataframe()      

## NFL

In [8]:
leagues_model.heatmap_counties("NFL")

leagues_model.render_map("NFL")

HTML(value='\n            <style>\n            .leaflet-popup-content-wrapper .leaflet-popup-tip {\n          …

Map(center=[38.72728229549864, -96.9010842308538], controls=(ZoomControl(options=['position', 'zoom_in_text', …

## MLB

In [9]:
leagues_model.heatmap_counties("MLB")

leagues_model.render_map("MLB")

HTML(value='\n            <style>\n            .leaflet-popup-content-wrapper .leaflet-popup-tip {\n          …

Map(center=[38.72728229549864, -96.9010842308538], controls=(ZoomControl(options=['position', 'zoom_in_text', …

## NBA

In [10]:
leagues_model.heatmap_counties("NBA")

leagues_model.render_map("NBA")

HTML(value='\n            <style>\n            .leaflet-popup-content-wrapper .leaflet-popup-tip {\n          …

Map(center=[38.72728229549864, -96.9010842308538], controls=(ZoomControl(options=['position', 'zoom_in_text', …

## NHL

In [11]:
leagues_model.heatmap_counties("NHL")

leagues_model.render_map("NHL")

HTML(value='\n            <style>\n            .leaflet-popup-content-wrapper .leaflet-popup-tip {\n          …

Map(center=[38.72728229549864, -96.9010842308538], controls=(ZoomControl(options=['position', 'zoom_in_text', …

## MLS

In [ ]:
leagues_model.heatmap_counties("MLS")

leagues_model.render_map("MLS")